In [1]:
import warnings
from langchain_google_genai import GoogleGenerativeAI
from crewai_tools import SerperDevTool
import os
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv
load_dotenv()
warnings.filterwarnings("ignore")

d:\Company Task\Task\Content-Generation-Workflow\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### search_tool and model for generating content and image


In [2]:
# Firstly, we set a search_tool and llm model
search_tool = SerperDevTool(n_results=5)

os.getenv("OPENAI_MODEL_NAME")

# Google gemini-pro model
llm = GoogleGenerativeAI(
    model="gemini-pro", google_api_key=os.getenv("GOOGLE_API_KEY"))

##### For image_generate search tool


In [3]:
import requests
import io
from PIL import Image
from crewai_tools import tool


@tool("Image Generate Tool")
def image_generate(context: str):
    """This is Image generate tool"""
    API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-xl-base-1.0"
    hugginface_api_key = os.getenv("HUGGINGFACE_API_KEY")
    headers = {"Authorization": f"Bearer {hugginface_api_key}"}

    def query(payload):
        response = requests.post(API_URL, headers=headers, json=payload)
        return response.content
    image_bytes = query({
        "inputs": context,
    })
    name = context.split(" ")[0]
    image = Image.open(io.BytesIO(image_bytes)).resize((1024, 1024)).save(f"../outputs/generate_{name}.jpg")
    return image

### Agents


In [4]:
researcher_agent = Agent(
    role="Researcher Agent",
    goal="Your goal is to assists in content research by gathering, analyzing, and summarizing information on specified topics.",
    backstory="""
    ### You are conducting keyword research and identifying trending topics relevant to the content strategy.
    - This involves using various tools and techniques to identify keywords and topics that are relevant to the content strategy of the organization.
    - Keyword research helps in understanding what topics are popular among the target audience and what keywords are commonly used in search queries related to the organization's industry or niche.
    - Identifying trending topics allows the organization to capitalize on current trends and create content that is timely and relevant.

    ### Also, you are collecting data and insights from reputable sources to support content creation.
    - This involves gathering data and information from reputable sources such as research papers, industry reports, authoritative websites, and credible publications.
    - The collected data and insights serve as valuable resources for content creation, providing facts, statistics, case studies, and other supporting evidence to strengthen the content's credibility and relevance.
    - Data collection may include conducting surveys, interviews, or market research to gather firsthand insights from target audiences or industry experts.

    ### Finally, you are proving summaries and briefs to human content creators to streamline their research phase.
    - This responsibility entails summarizing the gathered information and insights into concise and actionable briefs for human content creators.
    - Summaries and briefs help content creators quickly grasp the key points and main findings of the research, saving time and effort during the content creation process.
    - By providing organized and digestible summaries, the AI Research Assistant enables content creators to focus on creative ideation, storytelling, and content development rather than spending excessive time on research and information gathering.
    """,
    tools=[search_tool],
    verbose=True,
    max_iter=5,
)

content_creator_agent = Agent(
    role="Content Creator",
    goal="Your goal is to generate initial drafts of content based on templates, guidelines, and inputs for the research phase.",
    backstory="""
    - **Creating Initial Content Drafts for Various Formats:** This involves using natural language generation (NLG) technologies to generate preliminary drafts of content in different formats, such as blog posts, articles, and social media updates. NLG technologies use algorithms to analyze data and generate human-like text based on predefined rules, templates, or machine learning models. By leveraging NLG, content creators can quickly generate draft content that can then be refined and customized as needed.

    - **Adhering to Brand Voice, Style Guides, and Content Objectives:** Content creators must ensure that the generated content aligns with the brand's voice and adheres to established style guides and content objectives. This includes maintaining consistency in tone, language, and messaging across all content assets to reinforce the brand identity and messaging strategy. Content strategists may provide guidelines and objectives to ensure that the content effectively communicates the brand's values, resonates with the target audience, and supports broader marketing goals.

    - **Generating Creative Ideas for Content:** In addition to generating draft content, content creators are responsible for brainstorming and developing creative ideas for content campaigns. This includes crafting engaging headlines, taglines, and calls-to-action that capture the audience's attention and compel them to take action. Creativity is essential in devising unique and compelling content concepts that differentiate the brand from competitors and resonate with the target audience's interests and preferences.
    """,
    verbose=True
)

visual_content_creator_agent = Agent(
    role="Visual Content Creator",
    goal="Generates visual content such as images, infographics, and simple videos to complement textual content.",
    backstory="""
    - **Creating Visual Content Based on Textual Content Themes and Highlights:** Visual content creators translate textual content into engaging visuals, such as graphics, images, infographics, videos, or animations, to represent key themes and messages. This enhances audience engagement and understanding of the content.

    - **Adhering to Brand Visual Guidelines and Aesthetics:** Visual content must align with the brand's visual identity, using consistent colors, fonts, logos, and design elements. This maintains brand consistency and reinforces brand recognition across different platforms.

    - **Automatically Resizing and Adapting Visual Content for Different Platforms and Devices:** Visual content creators optimize assets for various screen sizes and platforms, ensuring they look great and perform well across devices. Techniques like responsive design and media queries are used to achieve this.
    """,
    tools=[image_generate],
    verbose=True
)


seo_analyst_agent = Agent(
    role="SEO Analyst",
    goal="Optimizes content for search engines and improves content discoverability online.",
    backstory="""
    - **Analyzing Content for SEO Best Practices:** Content analysts assess content to ensure it adheres to SEO best practices, including keyword density, meta descriptions, and title tags. This optimization enhances content visibility and ranking on search engine results pages (SERPs).

    - **Recommending Improvements to Enhance Content Ranking:** Content analysts suggest improvements based on their analysis to enhance content ranking. This includes suggesting additional keywords, refining meta descriptions, and optimizing title tags to increase relevance and authority in the eyes of search engines.
    
    - **Monitoring Content Performance and Providing Insights:** Content analysts continuously monitor content performance, including organic search traffic and user engagement metrics. They provide insights for content optimization by identifying underperforming content and recognizing successful content strategies for future campaigns. These insights drive continuous improvement in search engine visibility and performance.
    """,
    llm = llm,
    verbose=True
)

editorial_assistant_agent = Agent(
    role = "Editorial Assistant",
    goal= "Your goal is to implement a review process to check the content for accuracy, coherence, grammar, and style. Make necessary adjustments.",
    backstory="""
    - Detailed Grammar and Spelling Checks: The Editorial Assistant meticulously examines content to identify and rectify any grammatical or spelling errors. This involves scrutinizing each sentence and paragraph to ensure correctness in language usage.
    
    - Stylistic Adjustments:
    In addition to grammar and spelling, the Editorial Assistant makes stylistic adjustments to harmonize content with the publication's designated tone and style guidelines. This involves refining sentences, adjusting word choices, and ensuring consistency throughout the text.
    
    - Conducting Final Reviews:
    The Editorial Assistant performs thorough final reviews to verify the readiness of content for publication. This entails assessing all aspects of the content, including grammar, spelling, style, formatting, and adherence to publication standards.
    """,
    llm = llm,
    verbose=True
)

In [5]:
researcher_task = Task(
    description="""
     These responsibilities highlight the role of the AI Research Assistant in supporting the content creation process by leveraging AI and machine learning techniques to streamline research, gather relevant data and insights, and facilitate collaboration between AI and human content creators based on the user input {topic}.
    """,
    agent=researcher_agent,
    expected_output="""
    Structuring a content plan that includes headings, subheadings, and key points to cover for the {topic}.

    Exaple output:
    ### Title: The Role of AI in Healthcare

    ### Introduction:
    - Brief overview of ....

    ### 1: AI Applications in Diagnosis and Disease Detection
    * 1.1: Medical Imaging and Radiology
        * Key points:
            - Use of AI algorithms to analyze medical images (X-rays, MRIs, CT scans) for early detection of diseases.
            - Improved accuracy and efficiency in identifying abnormalities and diagnosing conditions such as cancer, fractures, and cardiovascular diseases.
    * 1.2: .......

    Also, give me some external website link.

    ### Conclusion:
    - Summary of the key roles and applications of AI in healthcare, from diagnosis and treatment to personalized medicine and remote monitoring.
    - Discussion of the potential benefits and challenges of AI adoption in healthcare, along with considerations for future developments.
    - Call-to-action encouraging readers to stay informed about advancements in AI healthcare and to engage in discussions about the ethical and regulatory implications.
    - Closing remarks emphasizing the transformative potential of AI in improving patient care and shaping the future of healthcare delivery.

    """,
    llm=llm
)

content_creator_task = Task(
    description="""
    The role of generating initial drafts of content requires a blend of creativity, research skills, attention to detail, and adherence to guidelines. By effectively translating research insights into compelling content drafts, content creators lay the groundwork for successful content marketing campaigns that resonate with the target audience and achieve business objectives.
    """,
    agent=content_creator_agent,
    expected_output="""
    - Based on the content plan, generate a draft using an AI text-generation model. This should form the body of your content.
    - Generating creative ideas for content, including headlines, taglines, and calls-to-action.
    """,
    llm=llm,
    context=[researcher_task]
)

visual_content_creator_task = Task(
    description="""
     Incorporate an AI-based image generation model to create relevant images for the content. Ensure these images are saved locally.
    """,
    agent=visual_content_creator_agent,
    expected_output="""Generate a .png file image based on the input and return it in output folder.""",
    tools = [image_generate],
    context=[content_creator_task],
)

seo_analyst_task = Task(
    description="""
    The role involves optimizing content for search engines by analyzing, recommending improvements, and monitoring performance, ultimately aiming to enhance content discoverability online and improve its ranking on search engine results pages.
    """,
    agent=seo_analyst_agent,
    expected_output="""
    Apply SEO best practices to the generated content. This may involve:
    - Keyword optimization.
    - Meta tags and descriptions.
    - Readability improvements.

    The expected output of an SEO Analyst is analyzing content for SEO best practices, including keyword density, meta descriptions, and title tags. Recommending improvements to enhance content ranking on search engines. Monitoring content performance and providing insights for content optimization.
    """,
    context=[content_creator_task],
)


editorial_assistant_task = Task(
    description="""
    The Editorial Assistant plays a pivotal role in guaranteeing that published content is error-free, stylistically appropriate, and prepared to meet the publication's standards and expectations.
    """,
    agent=editorial_assistant_agent,
    expected_output="""
    The expected output is high-quality content that is polished, accurate, and in line with the publication's editorial standards, ensuring a positive reader experience and maintaining the publication's reputation for excellence.
    """,
    context=[seo_analyst_task],
    output_file="../outputs/content_creator.md"
)


In [6]:
crew = Crew(
    agents=[researcher_agent, content_creator_agent,
            visual_content_creator_agent, seo_analyst_agent, editorial_assistant_agent],
    tasks=[researcher_task, content_creator_task, visual_content_creator_task, seo_analyst_task, editorial_assistant_task],
    process=Process.sequential
)

In [7]:
result = crew.kickoff(inputs={
    'topic': input("Enter the topic: ")
})



> Entering new CrewAgentExecutor chain...
I need to start by searching the internet for papers related to flash attention.

Action: Search the internet
Action Input: {"search_query": "papers on flash attention"} 


Search results: Title: FlashAttention: Fast and Memory-Efficient Exact Attention with IO ...
Link: https://arxiv.org/abs/2205.14135
Snippet: We argue that a missing principle is making attention algorithms IO-aware -- accounting for reads and writes between levels of GPU memory. We ...
---
Title: [2405.02803] Is Flash Attention Stable? - arXiv
Link: https://arxiv.org/abs/2405.02803
Snippet: Access Paper: View a PDF of the paper titled Is Flash Attention Stable?, by Alicia Golden and 10 other authors. View PDF · TeX Source · Other ...
---
Title: [PDF] Fast and Memory-Efficient Exact Attention with IO-Awareness
Link: https://openreview.net/pdf?id=H4DqfPSibmx
Snippet: ... flash-attention. 2. Page 3. We empirically validate that FLASHATTENTION speeds up model training and impr

In [8]:
result

'**Flash Attention: Revolutionizing Machine Learning**\n\nFlash attention has revolutionized the field of machine learning with its memory-efficient exact attention algorithm. By optimizing the allocation of computational resources during model training, flash attention incorporates IO-awareness to minimize memory reads and writes between GPU levels, enhancing model performance and efficiency.\n\n**Benefits of Flash Attention:**\n\n* Faster model training\n* Improved model quality due to reduced memory operations\n* Enhanced capacity to model longer context sequences\n* Increased accuracy of predictions\n\n**Practical Applications:**\n\n* Implementation of FlashAttention and FlashAttention-2 offers tools for researchers and developers\n* GitHub repositories and resources provide insights into the technical aspects and applications of flash attention in machine learning projects\n* Real-world case studies and applications demonstrate the effectiveness of flash attention in domains such 